# TIMESERIES FORECASTING - Autoregressive Integrated Moving Average (ARIMA)


## Install and import libraries
---
This exercise requires the `pdarima` library, which can be installed with Python's `pip` command. This command only needs to be done once per machine.

The standard, shorter approach may work:

In [ ]:
# pip install pmdarima --user

If the above command didn't work, it may be necessary to be more explicit, in which case you could run the code below.

In [ ]:
# import sys
# !{sys.executable} -m pip install pmdarima --user

Once `pdarima` is installed, then load the libraries below.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats                                 # needed for z-score
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from pmdarima.arima import auto_arima

## Load and prepare data
---

In [ ]:
df = pd.read_csv('../datasets/airpassengers.csv', parse_dates=['Month'], index_col=['Month'])

## Plot data
---

In [ ]:
fig, ax = plt.subplots()
plt.xlabel('Year: 1949-1960')
plt.ylabel('Monthly Passengers (1000s)')
plt.title('Monthly International Air Passengers')
plt.plot(df, color='black')
ax.xaxis.set_major_formatter(DateFormatter('%Y'))

## Split data into train and test set
---

### Create Training Dataset

In [ ]:
# Select data from before 1958.

train = df.loc[df.index < '1958-01-01']


In [ ]:
# Plot the training data

fig, ax = plt.subplots()
plt.xlabel('Year: 1949-1957')
plt.ylabel('Monthly Passengers (1000s)')
plt.title('Monthly International Air Passengers')
plt.plot(train, color='black')
ax.xaxis.set_major_formatter(DateFormatter('%Y'))

### Create Testing Dataset

In [ ]:
# Use data from 1958 through 1960 (the last three years) for testing.

test = df.loc[df.index >= '1958-01-01']

In [ ]:
# Plot the testing data.
# Note that the vertical scale changes from the previous graph.

fig, ax = plt.subplots()
plt.xlabel('Year: 1958-1960')
plt.ylabel('Monthly Passengers (1000s)')
plt.title('Monthly InInternational Air Passengers')
plt.plot(test, color='black')
ax.xaxis.set_major_formatter(DateFormatter('%Y'))

## Prepare the model
---

In [ ]:
# Test Stationarity
train_acf = plot_acf(train, lags=24)

### Linear Model

In [ ]:
lr = sm.OLS(endog=train['#Passengers'], exog=sm.add_constant(np.arange(1, 1 + train.shape[0]))).fit() 
print(lr.summary())

In [ ]:
# Extract the fitted values.
y_hat = lr.fittedvalues

In [ ]:
# Extract the 95% prediction interval.
y_ci = lr.get_prediction().conf_int(alpha=0.05)

In [ ]:
# Graph time series with linear regression line and corresponding 95% confidence interval.
fig, ax = plt.subplots()
plt.xlabel('Year: 1949-1957')
plt.ylabel('Monthly Passengers (1000s)')
plt.title('Monthly International Air Passengers')
plt.plot(train, color='black', label='Training Data')
plt.plot(y_hat, color='blue', label='Linear Regression Line')
plt.fill_between(y_hat.index, y_ci[:, 0], y_ci[:, 1], color='green', alpha=0.5, label='95% Conf. Int.')
plt.legend(bbox_to_anchor=(1.05, 1))
ax.xaxis.set_major_formatter(DateFormatter('%Y'))

## ARIMA: Train model
---

In [ ]:
auto_arima_model = auto_arima(train, m=12, with_intercept=False, suppress_warnings=True)
print(auto_arima_model.summary())

In [ ]:
# Extract the residuals. 
resid = auto_arima_model.resid()

In [ ]:
# Plot the standardized residuals.
fig, ax = plt.subplots()
plt.plot(train.index, stats.zscore(resid), color='gray') 
plt.title('Standardized Residuals')
plt.xlabel('Year: 1949-1957')
ax.xaxis.set_major_formatter(DateFormatter('%Y'))

In [ ]:
# Plot the ACF (autocorrelation function) of the residuals. 
res_acf = plot_acf(resid, lags=24)

## Test the model
---

In [ ]:
# Fit the best model to the training data.
auto_arima_model.fit(train)

In [ ]:
# Use the model to predict intervals for last three years.
# That is, apply the model to the testing dataset.
arima_predictions = auto_arima_model.predict(n_periods=36, alpha=0.05, return_conf_int=True)

In [ ]:
# Extract the time series of model predictions.
y_pred = pd.Series(arima_predictions[0], index=test.index)


In [ ]:
# Extract the 95% prediction interval.
y_pred_lb, y_pred_ub = arima_predictions[1][:, 0], arima_predictions[1][:, 1]

In [ ]:
# Graph the training data (1949-1957).
# Add the predictions for the testing data (1958-1960).
# Add observed values from testing data.

fig, ax = plt.subplots()
plt.xlabel('Year: 1949-1960')
plt.ylabel('Monthly Passengers (1000s)')
plt.title('Monthly International Air Passengers')
plt.fill_between(test.index, y_pred_lb, y_pred_ub, color='green', alpha=0.5, label='95% Conf. Int.')
plt.plot(train, color='black', label='Training Data')
plt.plot(test, color='blue', label='Test Data')
plt.plot(y_pred, color='red', label='Forecast')
plt.legend(bbox_to_anchor=(1.05, 1))
ax.xaxis.set_major_formatter(DateFormatter('%Y'))

In [ ]:
# Create a dataframe with the observed values for the testing dataset.
# Add the predictions from the ARIMA model.
# Add the low and high boundaries for the 95% confidence intervals.

test_pred = pd.DataFrame({
    'Actual': test.iloc[:, 0].values, 
    'Point Forecast': y_pred.values, 
    'Lo 95': y_pred_lb,
    'Hi 95': y_pred_ub
    }, index=test.index)

In [ ]:
test_pred.head()